# Подготовка данных


In [217]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split
%matplotlib inline 

In [218]:
data = pd.read_csv('titanic_train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [219]:
data.drop(['Name', 'Cabin','PassengerId','Ticket'],axis=1,inplace=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [220]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [221]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [222]:
data['Age'] = data['Age'].fillna(data['Age'].median())


In [223]:
data.dropna(axis = 0,inplace=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [224]:
data = pd.concat([data,
           pd.get_dummies(data['Pclass'],prefix='Pclass'),
           pd.get_dummies(data['Parch'],prefix='Parch'),
           pd.get_dummies(data['Embarked'],prefix='Embarked'),
           pd.get_dummies(data['Sex'],prefix='Sex'),
           pd.get_dummies(data['SibSp'],prefix='SibSp'),

          ],axis=1)


In [225]:
data.drop(['Pclass','Parch','Embarked','Sex','SibSp'],axis=1,inplace=True)

In [226]:
data.describe(include='all')

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,...,Embarked_S,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,...,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,29.315152,32.096681,0.240720,0.206974,0.552306,0.760405,0.132733,0.089989,0.005624,...,0.724409,0.350956,0.649044,0.681665,0.235096,0.031496,0.017998,0.020247,0.005624,0.007874
std,0.486260,12.984932,49.697504,0.427761,0.405365,0.497536,0.427077,0.339477,0.286327,0.074826,...,0.447063,0.477538,0.477538,0.466093,0.424297,0.174752,0.133018,0.140925,0.074826,0.088435
min,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,22.000000,7.895800,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,28.000000,14.454200,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,35.000000,31.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,80.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [227]:
data.isna().sum()

Survived      0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Sex_female    0
Sex_male      0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
dtype: int64

In [228]:
X = data.drop('Survived',axis=1)
X

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,...,Embarked_S,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,22.0,7.2500,0,0,1,1,0,0,0,0,...,1,0,1,0,1,0,0,0,0,0
1,38.0,71.2833,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,26.0,7.9250,0,0,1,1,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
3,35.0,53.1000,1,0,0,1,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
4,35.0,8.0500,0,0,1,1,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,27.0,13.0000,0,1,0,1,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
887,19.0,30.0000,1,0,0,1,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
888,28.0,23.4500,0,0,1,0,0,1,0,0,...,1,1,0,0,1,0,0,0,0,0
889,26.0,30.0000,1,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [229]:
y = data['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 889, dtype: int64

In [232]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7)
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((622, 24), (622,), (267, 24), (267,))

# Дерево (DecisionTreeClassifier) 

In [234]:
tree = DecisionTreeClassifier().fit(X_train,y_train)

In [236]:
tree.score(X_train,y_train) , tree.score(X_test,y_test)

(0.9839228295819936, 0.797752808988764)

In [239]:
roc_auc_score(y_test,tree.predict_proba(X_test)[:,1])

0.7959001782531194

In [246]:
pd.Series(tree.predict_proba(X_test)[:,1],index=range(1,X_test.shape[0]+1),name='target').to_csv('sample_submission_tree.csv',
                                                                                                header=True,
                                                                                                index_label='Survived')

In [247]:
!sample_submission_tree.csv

# Дерево(DecisionTreeClassifier) с подбором параметров (GridSearchCV)

In [274]:
params_tree = {
    'max_depth':range(10,101,10),
    'min_samples_split':range(1,10,2),
    'min_samples_leaf':range(1,10,2),
}

In [275]:
tree_grid_seacrh = GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=params_tree , cv =10,n_jobs=-1)

In [279]:
tree_grid_seacrh.fit(X_train,y_train)


C:\Users\emely\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.79254992 0.79577573 0.79252432 0.81021505        nan
 0.79098822 0.78776242 0.79098822 0.79743984        nan 0.79103943
 0.78786482 0.79426523 0.79103943        nan 0.79738863 0.80222734
 0.79577573 0.80222734        nan 0.79743984 0.79421403 0.79743984
 0.79260113        nan 0.75563236 0.78136201 0.78448541 0.80056324
        nan 0.76840758 0.77324629 0.76843318 0.7813108         nan
 0.78302611 0.79587814 0.78625192 0.79749104        nan 0.80222734
 0.79900154 0.80061444 0.79738863        nan 0.79743984 0.79421403
 0.79260113 0.79421403        nan 0.76208397 0.78771121 0.7812596
 0.80053763        nan 0.77165899 0.778085   0.77647209 0.78453661
        nan 0.78942652 0.78947773 0.78947773 0.79103943        nan
 0.79900154 0.80061444 0.80222734 0.79900154        nan 0.79421403
 0.79582693 0.79743984 0.79421403        nan 0.7555555

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(10, 101, 10),
                         'min_samples_leaf': range(1, 10, 2),
                         'min_samples_split': range(1, 10, 2)})

In [299]:
tree_grid_seacrh.best_params_

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 9}

In [277]:
tree_grid_seacrh.score(X_train,y_train) , tree_grid_seacrh.score(X_test,y_test)

(0.9212218649517685, 0.8127340823970037)

In [280]:
roc_auc_score(y_test,tree_grid_seacrh.predict_proba(X_test)[:,1])

0.8067439096850861

In [281]:
pd.Series(tree_grid_seacrh.predict_proba(X_test)[:,1],index=range(1,X_test.shape[0]+1),name='target')\
.to_csv('sample_submission_tree_grid_seach.csv',header=True,index_label='Survived')

In [282]:
!sample_submission_tree_grid_seach.csv

# Случайный лес (RandomForestClassifier)

In [284]:
forest = RandomForestClassifier()

In [285]:
forest.fit(X_train,y_train)

RandomForestClassifier()

In [286]:
forest.score(X_train,y_train) , forest.score(X_test,y_test)

(0.9839228295819936, 0.8089887640449438)

In [287]:
roc_auc_score(y_test,forest.predict_proba(X_test)[:,1])

0.8563576945929886

In [289]:
pd.Series(forest.predict_proba(X_test)[:,1], index=range(1,X_test.shape[0]+1) , name='taget')\
.to_csv('sample_submission_forest.csv',header=True,index_label='Survived')

In [290]:
!sample_submission_forest.csv

# Случайный лес(RandomForestClassifier) с подбором параметров(GridSearchCV)

In [295]:
params_forest = {
    'max_depth':range(1,10,2),
    'n_estimators':range(10,151,15),
    'min_samples_split':range(1,10,2),
    'min_samples_leaf':range(1,10,2),
}

In [296]:
forest_grid_search = GridSearchCV(estimator=RandomForestClassifier(),param_grid=params_forest,cv=10,n_jobs=-1)


In [297]:
forest_grid_search.fit(X_train,y_train)

C:\Users\emely\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan ... 0.79736303 0.80378904 0.79897593]
  warnings.warn(


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 10, 2),
                         'min_samples_leaf': range(1, 10, 2),
                         'min_samples_split': range(1, 10, 2),
                         'n_estimators': range(10, 151, 15)})

In [300]:
forest_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 3,
 'min_samples_split': 5,
 'n_estimators': 100}

In [301]:
forest_grid_search.score(X_train,y_train) , forest_grid_search.score(X_test,y_test)

(0.8617363344051447, 0.8052434456928839)

In [302]:
roc_auc_score(y_test,forest_grid_search.predict_proba(X_test)[:,1])

0.8690730837789661

In [339]:
pd.Series(forest_grid_search.best_estimator_.feature_importances_).sort_values(ascending=False)

15    0.297835
16    0.180058
1     0.139293
0     0.093883
4     0.072329
2     0.052820
3     0.023669
14    0.023254
18    0.021150
5     0.019895
17    0.017958
6     0.016129
12    0.012020
13    0.009324
7     0.007225
20    0.005088
21    0.002707
23    0.002216
19    0.001123
10    0.001034
22    0.000988
11    0.000000
9     0.000000
8     0.000000
dtype: float64

In [304]:
pd.Series(forest_grid_search.predict_proba(X_test)[:,1],name='target',index=range(1,X_test.shape[0]+1))\
.to_csv('sample_submission_forest_grid_search.csv',index_label='Survived',header=True)

In [305]:
!sample_submission_forest_grid_search.csv